# PLEASE NOTE: multipolygon and in general multy-geometries are not supported!

In [17]:
%pylab inline
from osgeo import ogr

Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shape', 'table']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [18]:
# from external_code import clip_shp
from earthpy.clip import clip_shp

# notice that for renaming a layer in a geopackage also 
```
ogrinfo my_geopackage.gpkg -sql "ALTER TABLE layer_name RENAME TO new_layer"
```
should work - not tested!

actually you need more commands:
https://gis.stackexchange.com/questions/252782/remove-a-raster-from-geopackage

e.g. 

``` 
ogrinfo -sql "delete from gpkg_tile_matrix_set where table_name='raster2'" droptest.gpkg
ogrinfo -sql "delete from gpkg_tile_matrix where table_name='raster2'" droptest.gpkg
ogrinfo -sql "delete from gpkg_metadata_reference where table_name='raster2'" droptest.gpkg
ogrinfo -sql "delete from gpkg_contents where table_name='raster2'" droptest.gpkg
ogrinfo -sql "drop table raster2" droptest.gpkg

```
or maybe not (?)
https://www.gdal.org/drv_geopackage.html


In [19]:
pack = "/home/luca/Downloads/Copernicus.gpkg"
shape = "/home/luca/Downloads/Archivio/area_detail.shp"

outpack = pack
outpack = "/home/luca/Downloads/Copernicus2.gpkg"

conn = ogr.Open(pack)

layernames = [ ]

# next we do some filtering, not always needed.
for i in conn:
    name = i.GetName()
    if not (name.startswith(" Landf") ):
        layernames.append(name)

if "layer_styles" in layernames:
    layernames.remove("layer_styles") # remove if any

print(layernames)

[' Contacts', ' Geological_UnitsOk', ' Liear_Features', ' Linear_Features', ' Pyroclastic_Deposits']


In [20]:
import geopandas as gpd




clipper = gpd.GeoDataFrame.from_file(shape) #.geometry[0]
clipper

,OBJECTID,SHAPE_Leng,SHAPE_Area,geometry
0,1.0,249495.769968,3.886445e+09,POLYGON ((-658858.2790000001 342997.0221000002...


In [22]:
tables =  []
for l in layernames:
    print (l)
    table = gpd.GeoDataFrame.from_file(pack, layer= l)
    if len(table) == 0:
        continue
        
    tables.append(table)
#     newtab = gpd.overlay(table, clipper, how='intersection')
    newtab = clip_shp(table, clipper)
    
    
    newname = l
    
    newtab.to_file(outpack, driver="GPKG", layer=newname)

 Contacts


/run/media/luca/data/Code/planmap-notebooks-code.git/mapping_utils/notebooks/external_code.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  shp_sub['geometry'] = shp_sub.intersection(poly)


 Geological_UnitsOk
 Liear_Features
 Linear_Features
 Pyroclastic_Deposits


# if you need to apply or do something on the style table 

In [ ]:
styles = gpd.GeoDataFrame.from_file(pack, layer= "layer_styles")

In [ ]:
orig_names = styles.f_table_name.values
new_names = [n.split()[1] for n in orig_names]
new_names
styles["f_table_name"] = new_names


In [ ]:
styles.to_file(pack, driver="GPKG", layer="layer_styles")